In [ ]:
# convert bytes into int 
def decode12(a:list) -> list:
    b = []
    for i in range(int(len(a)/3)):
        t1 = ((a[3*i+0] >> 0) | (a[3*i+1] << 8)) & 0xfff
        t2 = ((a[3*i+1] >> 4) | (a[3*i+2] << 4)) & 0xfff
        b.append(t1)
        b.append(t2)
    # print(t1,t2)
    return b

# convert int into bytes
def encode12(b:list) -> list:
    c = []
    for i in range(int(len(b)/2)):
        t1 = b[2*i+0] & 0xff
        t2 = ((b[2*i+0] >> 8) & 0xf) | ((b[2*i+1] & 0xf) << 4)
        t3 = (b[2*i+1] >> 4) & 0xff
        c.append(t1)
        c.append(t2)
        c.append(t3)
    return c

# it will write to the file
# mode = 1 -> convert int to hex
# mode = any -> write as it is
def write_to_file(path:str, a:list,mode:int):
    with open(path,"w") as fd:
        for i in a:
            if(mode == 1): 
                print("%.2x" % (i), file=fd)
            else:
                print(i, file=fd)
                
    print("[+] write to",path)

# mode = 1 -> convert to int
# mode = 2 -> convert to complete hex string
# mode = any -> read as it is
def read_from_file(path:str, mode:int) -> list:
    a = []
    line = 1
    with open(path,"r") as fd:
        while(line):
            line = fd.readline()
            line = line.replace('\n','')
            if(line!=''):
                if(mode == 1):
                    a.append(int(line, 16))
                else:
                    a.append(line)
    return a

e[i] <- CBD

PRF(s,b) = SHAKE256(s||b,$8*64*\mu$)

$\hat t  = \hat A \times \hat s + \hat e$


In [ ]:
import random
from hashlib import sha3_512, shake_256
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.ntt_invntt import barrett_reduce
from ipynb.fs.defs.polyvec import poly_basemul_montgomery, poly_ntt, poly_add
from ipynb.fs.defs.matrix_hash import samplentt_suite

# vector of depth k
def vector_multiply(a:list, b:list, k:int):
    assert(len(a)==len(b))
    r = []
    t = []
    for i in range(k):
        t.append(poly_basemul_montgomery(a[i],b[i]))
    for i in range(256):
        temp = 0
        for j in range(k):
            temp = temp + t[j][i]
        temp = barrett_reduce(temp)
        r.append(temp)
    return r

def hash_G(c:bytes):
    h = sha3_512(c)
    gest = h.digest()
    return gest

def hash_PRF(mu:int, r:str, N:int):
    m = "%s%.2x" % (r,N)
    h = shake_256(bytes.fromhex(m))
    gest = h.digest(64*mu)
    return gest.hex()

def to_pos(a:list):
    r = []
    for i in a:
        temp = i + ((i >> 15) & 3329)
        r.append(temp)
    return r

def string_split(a:list) -> list:
    r = []
    for i in range(int(len(a)/2)):
        r.append(a[2*i]+a[2*i+1])
    return r



SEED = 123
random.seed(SEED)
KYBER_K = 3
N = 0
A = [[]]*KYBER_K*KYBER_K
d = random.randbytes(32)
rho = hash_G(d).hex()[:64]
sigma = hash_G(d).hex()[64:]
e = [[]]*KYBER_K
s = [[]]*KYBER_K
t = [[]]*KYBER_K
s_pos = [[]]*KYBER_K
t_pos=  [[]]*KYBER_K

# matrix sample ntt
for i in range(3):
    for j in range(3):
        A[i*3+j] = samplentt_suite(rho,j,i)

for i in range(KYBER_K):
    s[i] = (poly_ntt(cbd2_hash(sigma,N)))
    s_pos[i] = to_pos(s[i])
    N = N + 1

for i in range(KYBER_K):
    e[i] = (poly_ntt(cbd2_hash(sigma,N)))
    N = N + 1

# print(len(s[0]),s)
# print(len(e[0]),e)
# print(len(A[0:3][0]),A[0:3])

for i in range(KYBER_K):
    temp = vector_multiply(A[i*KYBER_K:i*KYBER_K+3], s, KYBER_K)
    t[i] = poly_add(temp, e[i])
    t_pos[i] = to_pos(t[i]) 

ek_PKE = []
dk_PKE = []
for i in range(KYBER_K):
    temp1 = encode12(t_pos[i])
    temp2 = encode12(s_pos[i])
    for j in temp1:
        ek_PKE.append(j)
    for j in temp2:
        dk_PKE.append(j)

print("ek_PKE", ek_PKE)
print(decode12(ek_PKE))
print("dk_PKE", dk_PKE)
print(decode12(dk_PKE))

d_hex = []
for i in range(16):
    d_hex.append(d.hex()[2*i:2*i+2])

# print(d.hex())
print(d_hex)
msg = string_split(random.randbytes(32).hex())
print(msg)
r_rand = string_split(random.randbytes(32).hex())
print(r_rand)

write_to_file("../sim_input/d_randomness.hex", d_hex, 0)
write_to_file("../sim_input/decryption_key.hex", dk_PKE, 1)
write_to_file("../sim_input/encryption_key.hex", ek_PKE, 1)
write_to_file("../sim_input/p_seed.hex", string_split(rho), 0)
write_to_file("../sim_input/message.hex", msg, 0)
write_to_file("../sim_input/r_randomness.hex", r_rand, 0)


ek_PKE [184, 243, 15, 209, 108, 92, 198, 92, 96, 103, 113, 125, 238, 72, 46, 213, 217, 100, 220, 248, 11, 23, 209, 149, 140, 1, 28, 226, 85, 195, 26, 5, 53, 213, 229, 57, 78, 56, 187, 90, 178, 192, 138, 249, 82, 96, 66, 54, 101, 129, 22, 75, 56, 129, 213, 211, 196, 252, 192, 21, 29, 101, 29, 144, 197, 55, 240, 235, 155, 119, 17, 83, 7, 148, 2, 106, 11, 84, 246, 200, 189, 96, 90, 140, 73, 195, 108, 249, 130, 142, 62, 75, 109, 91, 178, 145, 132, 185, 155, 195, 185, 73, 197, 39, 108, 227, 136, 93, 12, 25, 39, 226, 195, 60, 154, 236, 9, 148, 154, 37, 93, 211, 13, 114, 161, 159, 42, 100, 110, 200, 57, 31, 0, 130, 8, 118, 65, 39, 143, 114, 83, 167, 168, 198, 86, 187, 157, 201, 166, 58, 15, 19, 158, 115, 252, 179, 247, 87, 79, 153, 208, 162, 250, 22, 152, 227, 71, 107, 174, 96, 72, 161, 242, 105, 133, 100, 192, 67, 193, 97, 213, 188, 142, 39, 184, 71, 32, 68, 178, 30, 243, 158, 199, 7, 200, 103, 234, 50, 175, 203, 59, 178, 96, 92, 210, 180, 102, 37, 242, 104, 222, 34, 195, 199, 12, 134, 174, 